**Lending Club Case Study**
===========================
<hr style="border-color:Yellow;">

### Importing Required Libraries
<hr style="border-color:cyan;">

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Reading the data
-------------------

In [ ]:
loan_data = pd.read_csv("./data/loan.csv")

In [ ]:
#Getting the Top Records
loan_data.head()

In [ ]:
#Checking the Data Dimension
loan_data.shape

### Data Cleaning Process Initiated
<hr style="border-color:Green;">

In [ ]:
#Checking for null data
loan_data.isnull().sum()

In the loan data shet there multiple fields with null values, we need to process these null value as a part of the data cleaning.
These fields dont provide any infoirmation related to the loan and in any kind they are not use full to get any information, so removing these data from the data set.

In [ ]:
# Removing all the 'NA' data from loan data set
loan_data.dropna(axis=1,how='all',inplace=True)
loan_data.shape


In [ ]:
# Data set after cleaning the null valued fields
loan_data.head()

In [ ]:
# Checking for duplicate value in "member_id" column
loan_data['member_id'].duplicated().any()

There is no member have taken the more than 1 loan, so we can remove this column as this column dont provide any information regarding the loan defaulters

In [ ]:
# Removing "member_id" column
loan_data.drop(['member_id'],axis=1,inplace=True)
loan_data.head()

In [ ]:
# Checking for duplicate value in "id" column and removing it as well
loan_data['id'].duplicated().any()


In [ ]:
# Removing "id" column
loan_data.drop(['id'],axis=1,inplace=True)
loan_data.head()

In [ ]:
loan_data['policy_code'].duplicated().sum()

In [ ]:
# Removing "policy_code" and other column as all the records have same value
loan_data.drop(['policy_code','delinq_amnt','tax_liens','pub_rec_bankruptcies','collections_12_mths_ex_med','acc_now_delinq','pub_rec_bankruptcies'],axis=1,inplace=True)
loan_data.head()

Columns such as "id", "member_id", "url", "title", "emp_title", "zip_code", "last_credit_pull_d", "addr_state", doesnt contribute to the loan defaulting in any way due to irrelevant information. So removing them.

In [ ]:
loan_data.drop(["url", "title", "emp_title", "zip_code", "last_credit_pull_d", "addr_state","out_prncp_inv","total_pymnt_inv","funded_amnt", "delinq_2yrs", "revol_bal", "out_prncp", "total_pymnt", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d" , "chargeoff_within_12_mths", "mths_since_last_delinq", "mths_since_last_record"], axis = 1, inplace = True)
loan_data.head()

According to the request made, we have to analyze the significant information about who is likely to be a defualters. So, to find this we dont require the records for the customers who is not a defaulter and not yet fully paid, i.e the status 'current'.

In [ ]:
# Filtering the records whos status is in 'Fully Paid' and "Charged Off"
loan_data = loan_data[loan_data.loan_status != "Current"]
loan_data.loan_status.unique()

### Checking For Missing Values
<hr style="border-color:Red;">

In [ ]:
# The remaining data dimension is
loan_data.shape

In [ ]:
# Let's see which field have missing value in %
((loan_data.isnull().sum()/len(loan_data.index))*100).round(2)

It Shows that column 'desc' is having misisng values which is more than 15%. So we need to remove that

In [ ]:
# Droping the 'desc' column
loan_data.drop(['desc'],inplace=True,axis=1)

In [ ]:
# Now, lets find out the data type of the fields
loan_data.info()

In [ ]:
print("Mode : " + loan_data['emp_length'].mode()[0])

In [ ]:
loan_data.emp_length.value_counts()

In [ ]:
# Filling the missing values with value which is used in maximum
loan_data.emp_length.fillna(loan_data.emp_length.mode()[0], inplace = True)
loan_data.emp_length.isna().sum()

In [ ]:
loan_data.dropna(axis = 0, subset = ['revol_util'] , inplace = True)
loan_data.revol_util.isna().sum()

### Standardizing the data
<hr style="border-color:Green;">

- "revol_util" column although described as an object column, it has continous values. 
- So we need to standardize the data in this column
- "int_rate" and "term" are such column.
- "emp_length" < 1 year is assumed as 0 and 10+ years is assumed as 10


In [ ]:
loan_data.revol_util = pd.to_numeric(loan_data.revol_util.apply(lambda x : x.split('%')[0]))
loan_data.int_rate = pd.to_numeric(loan_data.int_rate.apply(lambda x : x.split('%')[0]))
loan_data.emp_length = pd.to_numeric(loan_data.emp_length.apply(lambda x: 0 if "<" in x else (x.split('+')[0] if "+" in x else x.split()[0])))

In [ ]:
loan_data.head()

### Finding Outliers
<hr style="border-color:olive;">

In [ ]:
# Checking t he Outliers for annual_inc
sns.boxplot(loan_data['annual_inc'])

It shows that an outlier is present in the data. So, removing them.
To understand and remove the outlier, we can check the qualntile variations of the "annual_inc"

In [ ]:
quantile_info = loan_data.annual_inc.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info

In [ ]:
# Removing the outlier using the threshold 95%
per_95_annual_inc = loan_data['annual_inc'].quantile(0.95)
loan_data = loan_data[loan_data.annual_inc <= per_95_annual_inc]

In [ ]:
sns.boxplot(loan_data.annual_inc)

### Visualizing The Categorical data
<hr style="border-color:olive;">

As we already have grade column, We can visualize the defaulter data and plotting for 'Charged Off' loan_status

### Using the Univariat

#### Analyzing the Grade data

In [ ]:
sns.countplot(x = 'loan_status', data = loan_data)

In [ ]:
loan_data.sub_grade = pd.to_numeric(loan_data.sub_grade.apply(lambda x : x[-1]))
loan_data['sub_grade'].head()

In [ ]:
sns.set_palette('colorblind')
sns.countplot(x = 'grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] , hue = 'sub_grade',data = loan_data[loan_data.loan_status == 'Charged Off'])

It shows that Grade B5 is the highest in defaulter list

In [ ]:
sns.countplot(x = 'grade', data = loan_data[loan_data.loan_status == 'Charged Off'], order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'])

#### Analyzing the Purpose data

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
ax.set(xscale = 'log')
sns.countplot(y ='purpose', data=loan_data[loan_data.loan_status == 'Charged Off'])


Purpose as 'debt_consolidation' is highest in defaulters

#### Analyzing Home ownership

For Hom ownership column there 3 columns in which the data is 'none', so replacing that with 'Other'

In [ ]:
loan_data['home_ownership'].replace(to_replace = ['NONE'],value='OTHER',inplace = True)
loan_data['home_ownership'].unique()

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
ax.set(yscale = 'log')
sns.countplot(x='home_ownership', data=loan_data[loan_data['loan_status']=='Charged Off'])

'Rented' house is the highest in defaulters

In [ ]:
#creating bins for int_rate,open_acc,revol_util,total_acc
loan_data['int_rate_groups'] = pd.cut(loan_data['int_rate'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])
loan_data['open_acc_groups'] = pd.cut(loan_data['open_acc'],bins = 5,precision =0,labels=['2-10','10-19','19-27','27-36','36-44'])
loan_data['revol_util_groups'] = pd.cut(loan_data['revol_util'], bins=5,precision =0,labels=['0-20','20-40','40-60','60-80','80-100'])
loan_data['total_acc_groups'] = pd.cut(loan_data['total_acc'], bins=5,precision =0,labels=['2-20','20-37','37-55','55-74','74-90'])
loan_data['annual_inc_groups'] = pd.cut(loan_data['annual_inc'], bins=5,precision =0,labels =['3k-31k','31k-58k','58k-85k','85k-112k','112k-140k'])

In [ ]:
loan_data.head()

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
plt.subplot(221)
sns.countplot(x='int_rate_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])
plt.xlabel('Interest Rate')
plt.subplot(222)
sns.countplot(x='emp_length', data=loan_data[loan_data.loan_status == 'Charged Off'])

where rate of interest is 13-17% the defaulters are higher and also where the emp_lentgth is 10 the defaulters are higher.
There for here we can assume by the data that the customer having em_length=>10 and interest rate in between 13 - 17% are seen to be defaulter, but will contineu to further data analysis to get to more clear factors for defaulter.

In [ ]:
fig, ax = plt.subplots(figsize = (7,5))
ax.set_yscale('log')
sns.countplot(x='open_acc_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])

The defaulters are more where the open_acc is in between 2-10

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
ax.set_yscale('log')
sns.countplot(x='total_acc_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])

defaulters are higher if the total_acc is in between 2-20

In [ ]:
sns.countplot(x='revol_util_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])

defaulters are higher if the revol_util is greater than 80

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.countplot(x='annual_inc_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])

defaulters are higher if teh annual_income in between 31-58 thousand

In [ ]:
sns.countplot(y='term', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters are higher if the term is 36

In [ ]:
sns.countplot(x='verification_status', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters are higher if the source is not verified

In [ ]:
fig,ax = plt.subplots(figsize = (10,8))
ax.set_yscale('log')
sns.countplot(x='inq_last_6mths', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters re higher if the inq in last 6 months are 0

In [ ]:
fig,ax = plt.subplots(figsize = (7,5))
ax.set_yscale('log')
sns.countplot(x='pub_rec', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters are higher if the pub_rec is 0

In [ ]:
# Analyzing by issues month and year
df_month_year = loan_data['issue_d'].str.partition("-", True)     
loan_data['issue_month']=df_month_year[0]                       
loan_data['issue_year']='20' + df_month_year[2]
loan_data.head()

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(221)
sns.countplot(x='issue_month', data=loan_data[loan_data['loan_status']=='Charged Off'])
plt.subplot(222)
sns.countplot(x='issue_year', data=loan_data[loan_data['loan_status']=='Charged Off'])

loan provided in the month of december is higher, and also seen that the highest number of defaulters are in 2011.

#### Analyzing installment, dti and loan_amnt 

In [ ]:
# Creating bins for analysis
loan_data['installment_groups'] = pd.cut(loan_data['installment'], bins=10,precision =0,labels=['14-145','145-274','274-403','403-531','531-660','660-789','789-918','918-1047','1047-1176','1176-1305'])
loan_data['funded_amnt_inv_group'] = pd.cut(loan_data['funded_amnt_inv'], bins=7,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k']) ## bin is starting from -35?
loan_data['loan_amnt_groups'] = pd.cut(loan_data['loan_amnt'], bins=7,precision =0,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k'])
loan_data['dti_groups'] = pd.cut(loan_data['dti'], bins=5,precision =0,labels=['0-6','6-12','12-18','18-24','24-30'])

In [ ]:
fig,ax = plt.subplots(figsize = (12,5))
ax.set_yscale('log')
sns.countplot(x='funded_amnt_inv_group', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters are higher if the salary is in between 5000 - 10000

In [ ]:
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='loan_amnt_groups', data=loan_data[loan_data['loan_status']=='Charged Off'])

In [ ]:
sns.countplot(x='dti_groups', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters are higher if teh dti is in between 12 - 18

In [ ]:
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='installment_groups', data=loan_data[loan_data['loan_status']=='Charged Off'])

defaulters are higher if teh instalment amount in between 145 - 274

### Now, let's consolidate our observations for Univariant analysis
<hr style="border-color:olive;">

With all the above analysis we can draw the conclusions as follows:
- Grade is B
- Sub Grade is B5 level
- Rented House
- taken loan for debt clearance
- Loan interest rate of 13 to 17%
- Loan amount in between 5000 to 10000
- Monthly installments in between 145 to 274
- Term of 36 months
- When loan status is not verified
- When loan inquery in last 6 month is 0
- Open accounts in between 2-10
- Total account in between 2-20
- Emplaoyment tenure is 10
- Income range in between 30000 to 50000
- dti in between 12 to 18
- derogatory public records is 0

### Using the Bi-Variat

#### Analyzing with Respect to Annual income

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='annual_inc', y='purpose', hue ='loan_status',palette="deep")
plt.show()

Customer having salary in between 60 to 70 thousand and purpose is home_improvement has higher chances of being defaulter

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='home_ownership', y='annual_inc', hue ='loan_status',palette="pastel")
plt.show()

Customer having salary in between 60 to 70 thousand and having mortgage on home has more chances of being defaulter

In [ ]:
sns.barplot(x = "annual_inc_groups", y = "loan_amnt", hue = 'loan_status', data = loan_data)

Customer having highest income has got large amount of loans. And also seen that the defaulters are more in number across all income groups

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='int_rate_groups', y='annual_inc', hue ='loan_status',palette="pastel")
plt.show()

Customer having annual_inc in between 60 to 80 k has more defaulters with 21 - 24% rate of interest

#### Analyzing Loan amount with other variables

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt_groups', y='int_rate', hue ='loan_status',palette="pastel")
plt.show()

Loand amount between 30 to 35 k with interest rate as 15 to 17.5% has highest number of charged Off

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt', y='purpose', hue ='loan_status',palette="pastel")
plt.show()

Customer purpose is small bussiness and loan amount in between 12 - 14 k has the highest number of defaulters

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt', y='home_ownership', hue ='loan_status',palette="pastel")
plt.show()

Customer having loan of amount 12 - 16 k and home is mortgaeg and other has been highest in defaulters

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt', y='grade', hue ='loan_status',palette="pastel", order=['A','B','C','D','E','F','G'])
plt.show()

Customer with grade F and loan amount in between 16 to 20 k has high chances of getting defaulter

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(221)
sns.barplot(data =loan_data,y='loan_amnt', x='emp_length', hue ='loan_status',palette="pastel")
plt.subplot(222)
sns.barplot(data =loan_data,y='loan_amnt', x='verification_status', hue ='loan_status',palette="pastel")


customer with longer working tenure got large amount of loan. it also shows that verified loan have higher amount of loan.

Customer is a defaulter when the loan amount is 12 - 14 k and the working tenure is 10, and also if the source is verified but the loan amount is more than 16k is a defaulter

#### Grade with respect to int rt

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='int_rate', y='grade', hue ='loan_status',palette="pastel", order=['A','B','C','D','E','F','G'])
plt.show()

G has the highest int.rate

In [ ]:
plt.tight_layout()
sns.catplot(data =loan_data,y ='int_rate', x ='loan_amnt_groups', hue ='loan_status',palette="pastel",kind = 'box')

Higher int. rate has higher defaulters.

In [ ]:
sns.catplot(x = 'term', y = 'loan_amnt', data = loan_data,hue = 'loan_status', kind = 'bar')

Long term loans have large loan amount, larger the loan amount larger the defaulter

#### Now, lets cosolidate for Bi-Variant analysis
<hr style="border-color:olive;">

With all the above analysis we can draw the conclusions as follows:
- Customer taking loan for 'home improvement' and having income in between 60k -70k
- Customer with home ownership as 'MORTGAGE and have income of 60-70k
- Customer whose rate of interest is in between 21-24% and having an income of 70k-80k
- Customer whose loan amount in range 30k - 35k and interest rate of 15-17.5 %
- Customer whose purpose for loan is 'small business' and the loan amount is greater than 14k
- Customer whose home ownership is 'MORTGAGE and have loan of 14-16k
- When grade is F and loan amount is between 15k-20k
- When employment length is 10 yrs and loan amount is 12k-14k 
- When the loan is verified and loan amount is above 16k
- For grade G and interest rate above 20%